<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/ITBA_nlp01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

## Merged models

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
from matplotlib import pyplot as plt
import os
import json
from datetime import datetime, date
from dateutil.parser import parse
from dotenv import load_dotenv

from NLP_tools import Cleaning_text, top_keywords, top_entities, get_topic_name, best_document, clean_all, topic_documents
from core.functions import *

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

In [6]:
from opensearch_data_model import Topic, TopicKeyword, News, os_client, TOPIC_INDEX_NAME, NEWS_INDEX_NAME
from opensearch_io import init_opensearch, get_news
from opensearchpy import helpers

In [4]:
from openai import OpenAI

### Inicializamos la base vectorial

In [7]:
init_opensearch()

2024-07-22 16:18:37.240 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


El índice Topic ya existe. Saltando inicialización de base de datos.
El índice News ya existe. Saltando inicialización de base de datos.


### Path

In [8]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-NLP/data/'

In [9]:
if PATH == os.environ.get('PATH_LOCAL'):
    client = OpenAI(api_key= os.environ.get('OPENAI_API_KEY'))
else:
    from google.colab import userdata
    client = OpenAI(api_key= userdata.get('OPENAI_API_KEY'))

### Obtenemos los topicos de Opensearch filtrados por fecha

In [19]:
# Obtener los tópicos filtrados por la fecha seleccionada
date_str = '2024-04-02'
topics = get_topics_opensearch(date_filter=date_str)
sorted([  (row['index'], row['name']) for row in topics if row['to_date'][:10] <= date_str  ])

[(0, 'Deportes: Fútbol, Copa Libertadores, Sudamericana.'),
 (1, 'Entretenimiento y espectáculos variados.'),
 (2, 'Economía argentina, inflación y dólar.'),
 (3, 'Crimen, violencia y delincuencia.'),
 (4, 'Nuevas funciones en aplicaciones populares.'),
 (5, 'Críticas de Milei a gobernadores.'),
 (6, 'Turismo en Semana Santa.'),
 (7, 'Noticias y actualidad en Argentina.'),
 (8, 'Homenaje a caídos en Malvinas.'),
 (9, 'Elecciones en Venezuela: María Corina Machado'),
 (10, 'Conflictos en Oriente Medio.'),
 (11, 'Fallecimiento de la esposa de golfista.'),
 (12, 'Política y conflictos en Argentina.'),
 (13, 'Epidemia de dengue y prevención.'),
 (14, 'Accidentes de tránsito y robos.'),
 (15, 'Guerra de Malvinas - Veteranos.'),
 (16, 'Pronóstico del tiempo para marzo.'),
 (17, 'Criptomonedas: precios y cotizaciones actuales.')]

In [22]:
db_topics = []
data_topics = {} 
for reg in topics:
    index = reg['index']
    name = reg['name']
    similarity_threshold = reg['similarity_threshold']
    create_at = format_date(reg['created_at'])
    from_date = reg['from_date'][:10]
    to_date = reg['to_date'][:10]
    title_best_doc = reg['title_best_doc']
    id_best_doc = reg['id_best_doc']

    db_topics.append([index, name, round(similarity_threshold, 4), create_at, from_date, to_date, title_best_doc, id_best_doc])
    data_topics[index] = [name,
                        title_best_doc,
                        reg['best_doc'],
                        reg['entities'],
                        reg['keywords'],
                        similarity_threshold,
                        reg['vector']
                        ] 

db_topics = sorted(db_topics)
# Convertir a DataFrame
topics_df = pd.DataFrame(db_topics, columns=["indice", "nombre", "umbral", "creado", "desde", "hasta", "titulo noticia mas cercana", "id noticia"]) 

In [23]:
topics_df 

,indice,nombre,umbral,creado,desde,hasta,titulo noticia mas cercana,id noticia
0,0,"Deportes: Fútbol, Copa Libertadores, Sudamericana.",0.9508,2024-07-22 15:53,2024-04-01,2024-04-02,Quilmes enfrenta a Central Córdoba de Santiago del Estero por la Copa Argentina,105596567
1,1,Entretenimiento y espectáculos variados.,0.9709,2024-07-22 15:53,2024-04-01,2024-04-02,"Jorge Lanata, internado: el conductor volvió a la Fundación Favaloro por un problema respiratorio",105579290
2,2,"Economía argentina, inflación y dólar.",0.7852,2024-07-22 15:53,2024-04-01,2024-04-02,El agro ingresó 1.501 millones de dólares en marzo,105580392
3,3,"Crimen, violencia y delincuencia.",0.9582,2024-07-22 15:53,2024-04-01,2024-04-02,Ladrón fue detenido a golpes por su víctima y la justicia lo avaló | Crónica | Firme junto al pueblo,105581091
4,4,Nuevas funciones en aplicaciones populares.,0.8009,2024-07-22 15:53,2024-04-01,2024-04-02,"La inteligencia artificial responde mejor si somos emocionales en nuestras preguntas, demuestra estudio",105641112
5,5,Críticas de Milei a gobernadores.,0.8282,2024-07-22 15:53,2024-04-01,2024-04-02,"Elisa Carrió: ""Si el presidente cree que millones de argentinos tienen dólares debajo del colchón que me venga a revisar""",105630158
6,6,Turismo en Semana Santa.,0.7560,2024-07-22 15:53,2024-04-01,2024-04-02,"Semana Santa en Córdoba: calificaron como ""excelente"" el movimiento turístico",105594937
7,7,Noticias y actualidad en Argentina.,0.7725,2024-07-22 15:53,2024-04-01,2024-04-02,"La autora de ""Alas de sangre"" anuncia el tercer libro de la serie Empíreo - Libros - Entrelíneas - Cadena 3 Argentina",105579826
8,8,Homenaje a caídos en Malvinas.,0.7844,2024-07-22 15:53,2024-04-01,2024-04-02,Acto por Malvinas: Javier Milei rendirá homenaje a los caídos y compartiría el evento con Villarruel,105629675
9,9,Elecciones en Venezuela: María Corina Machado,0.8450,2024-07-22 15:53,2024-04-01,2024-04-02,El Gobierno y Colombia dan por superadas las diferencias,105629773


### Cargamos noticias de dos chunks

In [37]:
# Read the parquet file | ( lotes de prueba )

df_params = {'0_1000':'0_1000_data.parquet',
             '1000_2000':'1000_2000_data.parquet',
             '2000_3000':'2000_3000_data.parquet',
             'df_joined':'df_joined_2024-04-01 00_00_00.parquet'
            }

chunk = '0_1000'
df_parquet_1 = pd.read_parquet(PATH+df_params[chunk])
data_1 = list(df_parquet_1['in__text'])

chunk = '1000_2000'
df_parquet_2 = pd.read_parquet(PATH+df_params[chunk])
data_2 = list(df_parquet_2['in__text'])

In [38]:
# Unificamos datos
df_parquet = pd.concat([df_parquet_1, df_parquet_2], ignore_index=True)

### Modelo 1

In [9]:
# Cargar modelo 1 
chunk = '0_1000'

topic_model_1 = BERTopic.load(PATH+f"modelos/bertopic_model_{chunk}")
topics_1 = np.load(PATH+f"modelos/topics_{chunk}.npy")
probs_1 = np.load(PATH+f"modelos/probs_{chunk}.npy")

# Cargar los embeddings 
docs_embedding_1 = np.load(PATH+f"modelos/topic_embeddings_{chunk}.npy")

### Modelo 2

In [10]:
# Cargar modelo 2
chunk = '1000_2000'

topic_model_2 = BERTopic.load(PATH+f"modelos/bertopic_model_{chunk}")
topics_2 = np.load(PATH+f"modelos/topics_{chunk}.npy")
probs_2 = np.load(PATH+f"modelos/probs_{chunk}.npy")

# Cargar los embeddings 
docs_embedding_2 = np.load(PATH+f"modelos/topic_embeddings_{chunk}.npy")

### Merge

In [11]:
# Combine all models into one
merged_model = BERTopic.merge_models([topic_model_1, topic_model_2])


## Resultados

In [16]:
print(f"Cantidad de tópicos modelo 1: {len(topic_model_1.get_topic_info())} (incluye topico -1)")
print(f"Cantidad de tópicos modelo 2: {len(topic_model_2.get_topic_info())} (incluye topico -1)")

Cantidad de tópicos modelo 1: 17 (incluye topico -1)
Cantidad de tópicos modelo 2: 18 (incluye topico -1)


In [17]:
print(f"Cantidad de tópicos modelo merge: {len(merged_model.get_topic_info())} (incluye topico -1)")

Cantidad de tópicos modelo merge: 19 (incluye topico -1)


#### Comparamos modelos ordenados por topicos

In [31]:
# Obtener documentos de cada tópico y modelo
topic_freq_1 = topic_model_1.get_topic_freq()
topic_freq_2 = topic_model_2.get_topic_freq()
topic_freq_m = merged_model.get_topic_freq()

# Ordenar los DataFrames por 'Topic'
df1 = topic_freq_1.sort_values(by='Topic').reset_index(drop=True)
df2 = topic_freq_2.sort_values(by='Topic').reset_index(drop=True)
df3 = topic_freq_m.sort_values(by='Topic').reset_index(drop=True)

# Renombrar las columnas 'Count' para diferenciar ambos DataFrames
df1 = df1.rename(columns={'Count': 'Count1'})
df2 = df2.rename(columns={'Count': 'Count2'})
df3 = df3.rename(columns={'Count': 'Merged'})

# Combinar los DataFrames por 'Topic'
df_combined = pd.merge(df1, df2, on='Topic', how='outer')
df_combined = pd.merge(df_combined, df3, on='Topic', how='outer')

df_combined

,Topic,Count1,Count2,Merged
0,-1,140.0,119.0,259
1,0,157.0,320.0,326
2,1,130.0,169.0,130
3,2,116.0,57.0,436
4,3,112.0,49.0,183
5,4,84.0,36.0,176
6,5,52.0,34.0,52
7,6,30.0,31.0,30
8,7,27.0,29.0,56
9,8,26.0,26.0,83


### Recuperar todos los topicos y sus etiquetas generadas por el modelo

In [50]:
# docs de 0-1000 primer chunk
# docs de 1000-2000 segundo chunk
docs = [i for i, x in enumerate(merged_model.topics_) if x == 14]
for i in docs:
    print(i, df_parquet.iloc[i]['in__title'])

136 Marzo fue el décimo mes consecutivo en romper récords de altas temperaturas globales
162 Último día del fin de semana largo: cuál es el pronóstico del tiempo para este martes
223 Emiten alerta naranja por viento y lluvia para este martes
254 Pronóstico: para mañana se espera un aumento de la temperatura y alerta por Zonda y sectores del Gan Mendoza
354 El tiempo en Santiago del Estero: máxima de 26ºC y probabilidad de precipitaciones para el inicio de abril
359 Inicio de semana fresco y con un cielo parcialmente nublado - MDZ Online
505 El cielo estará nublado y se esperan lluvias durante toda la jornada en Tucumán
517 Lunes caluroso en Misiones, con pronostico de lluvias y chaparrones aislados
907 Clima en Córdoba: fuerte descenso de la temperatura para el lunes, ¿llega la lluvia?
966 Alerta por viento y lluvia, este martes y miércoles, en Neuquén y Río Negro: habrá ráfagas de 120 km/h
991 🟠 Lanzaron una alerta naranja por fuertes vientos: qué provincias serán afectadas
1085 Prono

### Buscar topicos ingresando un texto en el modelo merged

In [35]:
topic_res = merged_model.find_topics("cambio climatico")
topic_res

([14, 2, 7, 5, 13], [0.5230975, 0.33653283, 0.2686169, 0.25594658, 0.2443951])